Notebook completo:

In [ ]:
# Instalación de las librerías necesarias (solo ejecuta esta celda una vez)
!pip install langchain langchain_ollama chromadb sentence-transformers langchain_huggingface langchain_chroma beautifulsoup4

# Importación de las librerías necesarias
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import requests
from bs4 import BeautifulSoup
import re


Paso 1: Cargar datos desde una página web

In [ ]:
# Cargar los datos desde una página WEB (Wikipedia)
url = 'https://en.wikipedia.org/wiki/Football'
response = requests.get(url)

# Comprobamos si la página se cargó correctamente
if response.status_code == 200:
    print("Página cargada correctamente")

    # Usamos BeautifulSoup para analizar el contenido HTML de la página
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraemos solo los párrafos (etiquetas <p>) de la página
    paragraphs = soup.find_all('p')

    # Filtramos el contenido relevante de los párrafos
    clean_text = ''
    for paragraph in paragraphs:
        # Extraemos el texto de cada párrafo, eliminamos los saltos de línea y los espacios extra
        clean_text += paragraph.get_text(separator=' ', strip=True) + ' '

    # Limpieza adicional para el texto
    # Elimina las referencias de texto como [1], [2], etc.
    clean_text = re.sub(r'\[.*?\]', '', clean_text)

    # Elimina los espacios extras
    clean_text = re.sub(r'\s+', ' ', clean_text)

    # Elimina posibles saltos de línea innecesarios
    clean_text = clean_text.replace("\n", " ")

    # Mostramos una parte del texto limpio (primeros 1000 caracteres)
    print("Texto de la página cargado:")
    print(clean_text[:1000])  # Muestra los primeros 1000 caracteres del texto limpio
else:
    raise Exception(f"Error al cargar la página: {response.status_code}")


Explicación del código:

**1. Instalación de librerías**: Instalamos las librerías necesarias para el procesamiento de texto, la creación de embeddings (vectorización), y la interacción con modelos de chat. Aquí usamos beautifulsoup4 para extraer el texto de una página web.

**2. Cargar datos desde Wikipedia**: Cargamos el contenido de la página de Wikipedia sobre el fútbol y usamos BeautifulSoup para extraer solo los párrafos (p). El texto extraído se limpia para eliminar referencias y saltos de línea innecesarios, dejándonos con solo el texto relevante.

Paso 2: Preparar el modelo de embeddings y la base de datos

In [ ]:
# Seleccionar el modelo de embeddings (vectorización del texto)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Crear una base de datos de Chroma para almacenar y buscar los embeddings
vector_store = Chroma(
    collection_name="rag1",  # Nombre de la colección de datos
    embedding_function=embeddings,  # Función de embeddings para convertir el texto en vectores
    persist_directory="./datasets",  # Directorio donde se guardará la base de datos
)

# Configurar el "retriever" para acceder a la base de datos
retriever = vector_store.as_retriever()

print("Base de datos y retriever configurados correctamente.")


Explicación del código:

**1. Embeddings**: Usamos el modelo de embeddings de sentence-transformers/all-mpnet-base-v2 para convertir el texto en vectores de alta dimensión que el modelo de IA pueda procesar.

**2. Base de datos Chroma**: Usamos Chroma como base de datos para almacenar estos embeddings. Esta base de datos nos permitirá realizar búsquedas de texto basado en el contenido vectorizado.

**3. Retriever**: El retriever es la herramienta que nos permitirá buscar el contenido dentro de la base de datos Chroma, basándonos en el texto de entrada.

Paso 3: Configurar el modelo de IA y la cadena RAG

In [ ]:
# Plantilla de conversación (prompt) que se usará para generar respuestas
conversation_template = """Please answer the following question based only on the given context:
{context}

Question: {question}
"""

# Crear el prompt a partir de la plantilla
prompt_template = ChatPromptTemplate.from_template(conversation_template)

# Configurar el modelo local de Ollama (cambiar a tu modelo disponible)
local_llm = "tinyllama"  # Asegúrate de que este modelo esté disponible en tu entorno
local_model = ChatOllama(model=local_llm)

# Crear la cadena de procesamiento (RAG Chain)
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # Retriver obtiene el contexto
    | prompt_template  # El template crea el prompt a partir del contexto y la pregunta
    | local_model  # El modelo genera la respuesta
    | StrOutputParser()  # Se procesa la salida generada por el modelo
)

print("Cadena RAG configurada correctamente.")


Explicación del código:

**1. Plantilla de conversación**: Creamos una plantilla de conversación que pide una respuesta basada solo en el contexto proporcionado.

**2. Modelo local**: Usamos ChatOllama con el modelo tinyllama (puedes cambiarlo si tienes otros modelos disponibles).

**3. Cadena de procesamiento RAG**: La cadena de procesamiento recibe el contexto de Chroma, lo convierte en un prompt, y luego pasa el prompt al modelo para generar una respuesta.

Paso 4: Ejecutar una pregunta y obtener la respuesta

In [ ]:
# Ejecutamos una pregunta para probar el sistema
result = qa_chain.invoke("What is football?")
print("Respuesta de la IA:")
print(result)

Explicación del código:

**1. Invocar la cadena RAG**: Hacemos una consulta (en este caso, "What is football?") y ejecutamos el sistema.

**2. Obtener la respuesta**: El modelo generará una respuesta basada en el contexto que le proporcionamos de la página web de Wikipedia.

**Conclusión:**

Este notebook te permite extraer texto de una página web (en este caso, Wikipedia sobre el fútbol), procesarlo, y luego usar ese contenido como contexto para un modelo de IA que genera respuestas basadas en el contexto.